In [38]:
# This cell comes from starting a new Kaggle notebook

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# This prints all of the file names in the input directory
#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [39]:
# This is needed if attempting to use TPU in Kaggle's environment
#!pip install seaborn

In [40]:
# CITATION: this cell is a part of the following notebook, that I slightly changed:
#
# Thite, S. (2023, October 9). UBC ovarian cancer subtype classification-CNN. Kaggle. 
# https://www.kaggle.com/code/sunilthite/ubc-ovarian-cancer-subtype-classification-cnn/notebook

import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import classification_report , confusion_matrix , accuracy_score , auc
from sklearn.model_selection import train_test_split

import cv2
#from google.colab.patches import cv2_imshow
from PIL import Image 
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Input, Dense,Conv2D , MaxPooling2D, Flatten,BatchNormalization,Dropout
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow_hub as hub 

In [41]:
# If wanting to use a TPU
# CITATION: this code is from:
# Holbrook, R. (2020, September 24). Getting started with tpus. Kaggle. 
# https://www.kaggle.com/code/ryanholbrook/getting-started-with-tpus 
"""
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None
"""

"\ntry:\n    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() \n    print('Running on TPU ', tpu.master())\nexcept ValueError:\n    tpu = None\n"

In [42]:
# If wanting to use a TPU
# CITATION: this code is from:
# Holbrook, R. (2020, September 24). Getting started with tpus. Kaggle. 
# https://www.kaggle.com/code/ryanholbrook/getting-started-with-tpus 
"""
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 
"""

'\nif tpu:\n    tf.config.experimental_connect_to_cluster(tpu)\n    tf.tpu.experimental.initialize_tpu_system(tpu)\n    strategy = tf.distribute.experimental.TPUStrategy(tpu)\nelse:\n    strategy = tf.distribute.get_strategy() \n'

**Data loading and preprocessing**

CITATION: this section of the notebook is a part of the following notebook, that I slightly changed:

Thite, S. (2023, October 9). UBC ovarian cancer subtype classification-CNN. Kaggle. 
https://www.kaggle.com/code/sunilthite/ubc-ovarian-cancer-subtype-classification-cnn/notebook


In [43]:
train_df = pd.read_csv("/kaggle/input/UBC-OCEAN/train.csv")

In [44]:
# The paths for the medical images
path_train = "/kaggle/input/UBC-OCEAN/train_images"
path_test = "/kaggle/input/UBC-OCEAN/test_images"
train_folder = os.listdir(path_train)
test_folder = os.listdir(path_test)

In [45]:
# The paths for the thumbnail images
path_train_copy = "/kaggle/input/UBC-OCEAN/train_thumbnails"
path_test_copy = "/kaggle/input/UBC-OCEAN/test_thumbnails"
train_folder_copy = os.listdir(path_train_copy)
test_folder_copy = os.listdir(path_test_copy)

print(len(train_folder_copy))
print(len(test_folder_copy))

513
1


In [46]:
train_df_tma = train_df[train_df['is_tma']==True]

In [47]:
train_df_no_tma = train_df[train_df['is_tma']==False]

In [48]:
train_df_no_tma['image_id_path'] = [f"{i}_thumbnail.png" for i in train_df_no_tma['image_id']]

In [49]:
train_df_tma['image_id_path'] = [f"{i}.png" for i in train_df_tma['image_id']]

In [50]:
train_thumbnails_folder = train_folder_copy

In [51]:
image_data = []
image_label = []
path = "/kaggle/input/UBC-OCEAN/train_thumbnails"
path1="/kaggle/input/UBC-OCEAN/train_images/"
for img , label in zip(train_df_no_tma['image_id_path'],train_df_no_tma['label']):
    image = Image.open("/kaggle/input/UBC-OCEAN/train_thumbnails/"+img)
    image = image.resize((600,600))
    image = image.convert("RGB")
    image = np.array(image)
    image_data.append(image)
    image_label.append(label)

for img , label in zip(train_df_tma['image_id_path'],train_df_tma['label']):
    image = Image.open("/kaggle/input/UBC-OCEAN/train_images/"+img)
    image = image.resize((600,600))
    image = image.convert("RGB")
    image = np.array(image)
    image_data.append(image)
    image_label.append(label)

In [52]:
set(image_label)

{'CC', 'EC', 'HGSC', 'LGSC', 'MC'}

In [53]:
image_label_1 = []
for i in image_label:
    if i=="CC":
        image_label_1.append(0)
    elif i=="EC":
        image_label_1.append(1)
    elif i=="HGSC":
        image_label_1.append(2)
    elif i=="LGSC":
        image_label_1.append(3)
    elif i=="MC":
        image_label_1.append(4)

In [54]:
x = np.array(image_data)
y = np.array(image_label_1)

In [55]:
x_train , x_test, y_train, y_test = train_test_split(x,y,test_size=0.15,shuffle=True)

In [56]:
x_train_scaled = x_train/255
x_test_scaled = x_test/255

In [57]:
x_train , x_test, y_train, y_test = train_test_split(x,y,test_size=0.15,shuffle=True)

In [58]:
# This is my addition
# In some runs, I would delete some of the arrays that are no longer needed
# This is to save space in the Kaggle environment as it is limited
import gc
del x
del y
del image_data
del image_label_1
del image_label
del train_df
del train_df_tma
del train_df_no_tma
gc.collect()

811

**Experiments**

In [59]:
from keras.applications import EfficientNetB7
from keras import layers

# This is the function that creates a model of the third architecture, that is, EfficientNet
# To change the version of EfficientNet, I manually changed the first import in this cell and the constructor of the EfficientNet
# I also needed to change the input size based on which EfficientNet I used
#
# CITATION: I used code from the Keras website for this function:
# Fu, Y. (2020, June 30). Keras documentation: Image Classification via fine-tuning with EfficientNet. Keras. 
# https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/ 
def getModelPretrained():
    inputs = layers.Input(shape=(600, 600, 3))
    model = EfficientNetB7(include_top=False, input_tensor=inputs, weights="imagenet", input_shape=(224,224,3))
    
    # Freeze the pretrained weights
    model.trainable = False
    
    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(5, activation="softmax", name="pred")(x)

    # Compile
    model = keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(
        optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    
    return model

In [ ]:
# This is my code for experimenting with an EfficientNet model
# The part of the code in this cell that is not mine is the 5-fold cross validation
# CITATION: that part of the code is from:
#
# Versloot, C. (2020, June 11). how-to-use-k-fold-cross-validation-with-keras.md. GitHub. 
# https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-k-fold-cross-validation-with-keras.md 

from sklearn.model_selection import KFold
import numpy as np
import pandas as pd

epoch = 5

# This is the code from the citation above
acc_per_fold = []
loss_per_fold = []

x_values = np.concatenate((x_train_scaled, x_test_scaled), axis=0)
y_values = np.concatenate((y_train, y_test), axis=0)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)
# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(x_values, y_values):
    # This is to utilize the TPU
    # The next two lines after this one need to be indented to use this
    # This is from the citation for the TPU usage at the beginning of this notebook
    #with strategy.scope():
    model = getModelPretrained()

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(x_values[train], y_values[train],
              batch_size=64,
              epochs=epoch)

    # Generate generalization metrics
    scores = model.evaluate(x_values[test], y_values[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1
    
    # From here it is back to my own code
    del model
    del history
    gc.collect()

average_accuracy = np.mean(acc_per_fold)
average_loss = np.mean(loss_per_fold)

# I did not save the results into a file in this notebook as I trained one model at a time for EfficientNet
# I wrote down the results from this manually

# I did not include the output for this cell as it is long

In [ ]:
# Can uncomment these to print the resulting cross-validation accuracy and loss
#print("Average validation accuracy: " + str(average_accuracy))
#print("Average validation loss: " + str(average_loss))